(Part2_Initial_Reprojections)=
# Part 2: Initial Reprojections

```{tableofcontents}
```

This notebook focuses on taking all major inputs and reprojecting and clipping them to the same region of interest.  Most importantly, everything is reprojected to EPSG 32613, which is the UTM Grid Zone for Denver.

As usual, I'm totally hacking here, so please forgive my terrible habits.

## General Outline

1. Import Packages
2. Load configuration data
3. Parse collection folder for every collect ID.
   1. Verify pathname is valid ID, then parse components
   2. Check what files exist
4. Reproject all Landsat images to EPSG 32613 and crop to ROI

## Step 1: Import Required Packages

In [1]:
import configparser, datetime, logging, os, sys, time
from pyproj import CRS, Transformer
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import rasterio
from rasterio.plot import show, show_hist
from rasterio.windows import Window

from tqdm import notebook as tqdm

from skimage import exposure

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
#  DUG API
sys.path.insert(0,'..')
import dug_api.coordinate as dug_crd
import dug_api.imagery as dug_img
from dug_api.CollectID import CollectID, FileType
import dug_api.config as dug_cfg
import dug_api.wms as dug_wms

**Note:** If this flag is set to `True`, then no images will be written if a file already exists at the path.

In [3]:
skip_write_if_exists = True
update_if_duplicate  = False
filter_rasterio_logs = True

Setup logger.  If you are debugging, use `logging.DEBUG` rather than `logging.INFO`.

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Notebook')
logger.info( f'This Notebook run at {datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")}' )
if filter_rasterio_logs:
    logging.getLogger('rasterio').setLevel(logging.INFO)
    logging.getLogger('matplotlib').setLevel(logging.INFO)

pd.set_option('display.max_colwidth', None)

INFO:Notebook:This Notebook run at 2024:01:05 15:02:51


### Note to User

Given the quirks of Jupyter-Book in generating this webpage, an environment variable is utilized to allow skipping modifying the database.  This is done so the CM version, built on my own machine, can be used without modification.

## Step 2:  Loading Configuration Data

I've done this long enough to know that any defined variable which is math or coordinate related needs to be in a configuration file.   This project includes `options.cfg`, which stores all settings for this project.

In [5]:
config = Configuration( '../data/options.cfg' )
logger.info( f'Is Configuration Valid:  {config.is_valid()}' )

INFO:Notebook:Is Configuration Valid:  True


We need to define the region we wish to process.  Not every image is guaranteed to be at the same projection, with the same footprint.  This means we need a globally defined bounding box, plus coordinate reference system (CRS).


In [6]:
logger.info( f'Output EPSG Code: {config.get_output_epsg()}' )

INFO:Notebook:Output EPSG Code: 32613


An EPSG code 32613 means a Universal Transverse Mercator projection (326) centered on Grid Zone 13.  
```{figure} ../images/grid_zones.png
:name: utm_zones
:alt: Standard UTM Grid Zones
:class: bg-primary mb-1
:width: 80%
:align: center
```


## Step 3:  Build Initial Collection List

Given the coordinate information, we can start finding imagery datasets. 

We have a directory listed with all the collections provided.

In [7]:
collection_dir = config.get_image_collection_path()
folder_list = [ x[0] for x in os.walk( collection_dir )][1:]
folder_list = list(filter( lambda x: '.git' not in x, folder_list ))
logger.info( f'Located {len(folder_list)} possible collection folders.' )
pd.DataFrame( { 'folders': folder_list } ).head(5)

INFO:Notebook:Located 1481 possible collection folders.


,folders
0,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02
1,/Volumes/data/imagery/Landsat/collections/LE07_L1GT_033033_20090822_20200911_02
2,/Volumes/data/imagery/Landsat/collections/LE07_L1GT_033033_20090822_20200911_02/gap_mask
3,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02
4,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02/gap_mask


Landsat collections have a very specific naming structure.  See {cite}`landsat_ard_ctrl_book_2023`.


```{figure} ../images/naming_conventions.png
:name: LS_L1TP_naming
:alt: Landsat Level 1 Collection Product Naming
:class: bg-primary mb-1
:width: 80%
:align: center
```

```{figure} ../images/ard_naming_conventions.png
:name: LS_ARD_naming
:alt: Landsat ARD Product Naming
:class: bg-primary mb-1
:width: 80%
:align: center
```

This is a figure from page 32.

Now we can take the folders and verify which ones have valid collection IDs.  For this, we have the `CollectID` class in the `dug_api` package.


In [8]:
valid_cids = []
for folder in folder_list:

    cid = CollectID.from_pathname( folder )
    if cid != None:
        valid_cids.append( cid )
new_df = CollectID.list_to_dataframes( valid_cids )
new_df.head(5)

,pathname,cid,sensor,satellite,product_type,processing_level,wrs2_path,wrs2_row,ard_col,ard_row,...,B7_path,B8_path,B9_path,B10_path,B11_path,BT_B6_path,ST_B6_path,ST_CDIST_path,ST_EMIS_path,ST_QA_path
0,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02,LE07_CU_012009_20130529_20210501_02,E,7,ARD,ARD_CU,NaN,NaN,12.0,9.0,...,None,None,None,None,None,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_BT_B6.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_B6.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_CDIST.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_EMIS.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_QA.TIF
1,/Volumes/data/imagery/Landsat/collections/LE07_L1GT_033033_20090822_20200911_02,LE07_L1GT_033033_20090822_20200911_02,E,7,L1GT,L1GT,33.0,33.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02,LE07_L1TP_033032_20130105_20200908_02,E,7,L1TP,L1TP,33.0,32.0,NaN,NaN,...,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02/LE07_L1TP_033032_20130105_20200908_02_T1_B7.TIF,None,None,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02/LE07_L1TP_033032_20130105_20200908_02_T1_B6_VCID_2.TIF,None,None,None,None,None,None
3,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033033_20090110_20200912_02,LE07_L1TP_033033_20090110_20200912_02,E,7,L1TP,L1TP,33.0,33.0,NaN,NaN,...,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033033_20090110_20200912_02/LE07_L1TP_033033_20090110_20200912_02_T1_B7.TIF,None,None,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033033_20090110_20200912_02/LE07_L1TP_033033_20090110_20200912_02_T1_B6_VCID_2.TIF,None,None,None,None,None,None
4,/Volumes/data/imagery/Landsat/collections/LE07_L1GT_033033_20090502_20200912_02,LE07_L1GT_033033_20090502_20200912_02,E,7,L1GT,L1GT,33.0,33.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


Now we need to join the tables and update the database

In [9]:
collection_df = config.update_table( 'collection_list', new_df, update_file=True )  
collection_df.head(5)

,pathname,cid,sensor,satellite,product_type,processing_level,wrs2_path,wrs2_row,ard_col,ard_row,...,B7_path,B8_path,B9_path,B10_path,B11_path,BT_B6_path,ST_B6_path,ST_CDIST_path,ST_EMIS_path,ST_QA_path
0,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02,LE07_CU_012009_20130529_20210501_02,E,7,ARD,ARD_CU,NaN,NaN,12.0,9.0,...,None,None,None,None,None,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_BT_B6.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_B6.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_CDIST.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_EMIS.TIF,/Volumes/data/imagery/Landsat/collections/LE07_CU_012009_20130529_20210501_02/LE07_CU_012009_20130529_20210501_02_ST_QA.TIF
1,/Volumes/data/imagery/Landsat/collections/LE07_L1GT_033033_20090822_20200911_02,LE07_L1GT_033033_20090822_20200911_02,E,7,L1GT,L1GT,33.0,33.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02,LE07_L1TP_033032_20130105_20200908_02,E,7,L1TP,L1TP,33.0,32.0,NaN,NaN,...,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02/LE07_L1TP_033032_20130105_20200908_02_T1_B7.TIF,None,None,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033032_20130105_20200908_02/LE07_L1TP_033032_20130105_20200908_02_T1_B6_VCID_2.TIF,None,None,None,None,None,None
3,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033033_20090110_20200912_02,LE07_L1TP_033033_20090110_20200912_02,E,7,L1TP,L1TP,33.0,33.0,NaN,NaN,...,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033033_20090110_20200912_02/LE07_L1TP_033033_20090110_20200912_02_T1_B7.TIF,None,None,/Volumes/data/imagery/Landsat/collections/LE07_L1TP_033033_20090110_20200912_02/LE07_L1TP_033033_20090110_20200912_02_T1_B6_VCID_2.TIF,None,None,None,None,None,None
4,/Volumes/data/imagery/Landsat/collections/LE07_L1GT_033033_20090502_20200912_02,LE07_L1GT_033033_20090502_20200912_02,E,7,L1GT,L1GT,33.0,33.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [10]:
collection_df.loc[collection_df['cid'] == 'LC09_L1TP_034032_20230516_20230517_02_T1']

,pathname,cid,sensor,satellite,product_type,processing_level,wrs2_path,wrs2_row,ard_col,ard_row,...,B7_path,B8_path,B9_path,B10_path,B11_path,BT_B6_path,ST_B6_path,ST_CDIST_path,ST_EMIS_path,ST_QA_path


### Setup Landsat Config Files


In [11]:
for cr in config.ls_collection_df.itertuples():
    logger.debug( f'Testing CID: {cr.cid}' )
    cid_config = config.get_ls_collection_config( cr.cid )

    #  make sure a few key fields are there
    assert( cid_config.has_option('general','cid') )
    assert( cid_config.has_option('general','pathname') )

INFO:root:Building new ls config for cid: 0    LE07_CU_012009_20130529_20210501_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 1    LE07_L1GT_033033_20090822_20200911_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 2    LE07_L1TP_033032_20130105_20200908_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 3    LE07_L1TP_033033_20090110_20200912_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 4    LE07_L1GT_033033_20090502_20200912_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 5    LE07_CU_011009_20120906_20210501_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 6    LE07_L1TP_033033_20121017_20200908_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 7    LE07_CU_012009_20080727_20210430_02
Name: cid, dtype: object
INFO:root:Building new ls config for cid: 8    LE07_CU_011009_20110927_20210501_02
Name: cid, dtype: object
INFO:root:Building

## Step 4:  Reproject all relevant images

As previously mentioned, Landsat data comes in different sizes, projections, and representations.  We need to reproject all images into the same coordinate system.


We need to get polygon points in UTM.

The order in the config is `[[min x, min y]],[max x, max y]]`

In [12]:
bbox_utm = config.get_region()
logger.info( bbox_utm )

INFO:Notebook:[[462191.7886451932, 4357374.143059055], [547590.5223242528, 4450167.255409848]]


Points: [[-105.438889, 39.365], [-104.440833, 40.200556]]


1. Iterate over each collection.
2. Iterate over each file in the collection (`file_<key>`).
3. Get the reprojected path and see if it exists yet.

In [13]:
file_types_to_reproject = [ f'b{x}_path' for x in range( 1, 12 ) ]

In [14]:
entries = { 'Collect-ID': collection_df['cid'].unique() }
for f in file_types_to_reproject:
    entries[f] = ''

completed_rows = pd.DataFrame( entries )

In [15]:
dest_epsg = config.get_output_epsg()
dest_gsd  = config.get_output_gsd()

pbar_cid = tqdm.tqdm( total=collection_df.shape[0] )
cid_counter = 0

pbar_files = tqdm.tqdm( total = len(file_types_to_reproject) )
file_counter = 0

#  iterate over each row
for cr in config.ls_collection_df.itertuples():

    cr_dict = cr._asdict()
    cid = cr_dict['cid']

    #  Clear the progress bar
    pbar_files.reset()

    #  Get the collection configuration object to update
    cid_config = config.get_ls_collection_config( cid )
    
    #  iterate over each file we need to reproject
    for col in file_types_to_reproject:

        update_db = False
        
        #  Skip if the collection doesn't have the band
        if cid_config.has_option('general',col) == False:
            pbar_files.update( 1 )
            continue
        path_in  = cid_config.get('general',col)

        path_out_key = f'{col}_epsg_{dest_epsg}'
        if cid_config.has_option('general',path_out_key):
            path_out = cid_config.get('general',path_out_key)
        else:
            in_bname = os.path.splitext(os.path.basename( path_in ) )
            path_out = os.path.join( os.path.dirname( path_in ), f'{in_bname[0]}.epsg_{dest_epsg}{in_bname[1]}' )
            cid_config['general'][path_out_key] = path_out
            update_db = True

        # if there is no input path, simply skip it (Each dataset only has a subset of possible types)
        if path_in is None:
            pass

        #  This is slightly concerning.  This is the input path is defined, but not found.
        elif not os.path.exists( path_in ):
            raise Exception( f'Source image {path_in} does not exist.' )
        
        elif skip_write_if_exists and path_out != None and os.path.exists( path_out ):
            completed_rows.loc[completed_rows['Collect-ID'] == cid, col] = 'Skipped'
        
        else:
                                         
            # Reproject image
            dug_img.reproject_image( path_in  = path_in,
                                     path_out = path_out,
                                     epsg     = dest_epsg,
                                     bbox_utm = bbox_utm,
                                     dest_gsd = dest_gsd )

            #  Note where the new reprojected file is (Keep this indent level as this file even
            #  if the processing is skipped
            update_db = True
            
            completed_rows.loc[completed_rows['Collect-ID'] == cid, col] = 'Completed'

        #  Write intermediate results to disk
        if update_db:
            config.set_ls_collection_config( cid_config )

        pbar_files.update( 1 )
        
    pbar_cid.update( 1 )

  0%|          | 0/1087 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

NameError: name 'imagery' is not defined

In [ ]:
completed_rows

#  Visualizing Results


For this analysis, we'll use the City Lights Garden as our reference image. 

In [ ]:
garden_name = 'City Lights at Benedict Park Place Community Garden'

lon_key = 'Garden Geolocation (Longitude)'
lat_key = 'Garden Geolocation (Latitude)'

def get_garden():
    csv_path = '../data/GardenDataForMaps120623.xlsx'
    all_gardens_df = pd.read_excel( csv_path )
    garden_df = all_gardens_df.loc[all_gardens_df['Garden: Garden Name'] == garden_name]
    return garden_df

garden_df = get_garden()
assert( garden_df.shape[0] == 1 )
garden_df

#### NAIP
The National Agriculture Image Program (NAIP) provides < 0.25 m GSD imagery for most of CONUS.  This is the highest-resolution and most up-to-date imagery you will find.

In [ ]:
garden_ll = ( garden_df[lon_key].values[0],
              garden_df[lat_key].values[0] )

garden_roi = dug_crd.convert_coord_roi( center_coord       = garden_ll,
                                        input_epsg         = 4326,
                                        window_size_pixels = [800,800],
                                        dest_gsd           = 1,
                                        output_epsg        = config.get_output_epsg() )

In [ ]:
wms_gsd = 1.0
wms_window_m = np.array( [600,600], np.float32 )
wms_window_pix = list(wms_window_m / wms_gsd)
wms_image = load_tile( base_url     = config.get('naip','wms_url'),
                       epsg         = config.get_output_epsg(),
                       center_ll    = garden_ll,
                       win_size_pix = wms_window_pix,
                       gsd          = wms_gsd,
                       layers       = [config.get('naip','wms_layers')],
                       format       = config.get('naip','wms_format') )
wms_image = np.transpose( wms_image, axes = [1, 2, 0 ] )[:,:,:3]

## Landsat 7 and 8 Collections

Landsat 7: `LE07_L1TP_033032_20130801_20200907_02_T1`
Landsat 8: `LC08_L1TP_033032_20230805_20230812_02_T1`

To visualize the imagery, we load the Blue, Green, and Red bands. For plotting, I recommend using `rasterio.plot` in memory-intensive applications as well as test applications.  It's not the prettiest, such as plotly, but it works really efficiently.  See [link](https://rasterio.readthedocs.io/en/stable/topics/plotting.html) for examples.

In [ ]:
analysis_cid7 = 'LE07_L1TP_033032_20130801_20200907_02_T1'
analysis_cid8 = 'LC08_L1TP_033032_20230805_20230812_02_T1'

In [ ]:
def load_and_merge_frames( cid, bands, roi, epsg, equalize = False, perform_transpose=False ):

    cid_config = config.get_ls_collection_config( cid )

    images = []
    for id in bands:

        key    = f'b{id}_path_epsg_{epsg}'
        driver = rasterio.open( cid_config.get('general',key) )

        ls_window = dug_img.get_window_from_coords( roi[0], roi[1], epsg, driver )

        image = driver.read(1, window=ls_window)

        if driver.profile['dtype'] == 'uint16':
            image = np.clip( image.astype('uint16') / 128, 0, 255 )

        # https://stackoverflow.com/a/47398865/2142228
        if equalize:
            image = exposure.adjust_log( image, 1)
        images.append( image )

    output = np.dstack( images ).astype('uint8')
    if perform_transpose:
        output = np.transpose( output, axes=[0,1,2] )
        
    return output

In [ ]:
ls7_rgb_image = load_and_merge_frames( analysis_cid7, [3,2,1], garden_roi, config.get_output_epsg(), True )
ls8_rgb_image = load_and_merge_frames( analysis_cid8, [4,3,2], garden_roi, config.get_output_epsg(), False )

fig_ov = make_subplots( rows=3, cols=2, 
                        subplot_titles=['NAIP',      'Histogram',
                                        'Landsat 7', 'Histogram',
                                        'Landsat 8', 'Histogram'], 
                        horizontal_spacing=0.05,
                        vertical_spacing=0.05,
                        column_widths=[0.5,0.5])

#  Add WMS Image
fig_ov.add_trace( go.Image( z = wms_image ), row=1, col=1 )
counts, bins = np.histogram( wms_image.flatten(), bins = 30 )
bins = 0.5 * (bins[:-1] + bins[1:])

fig_ov.add_trace( go.Scatter( x = bins,
                              y = counts,
                              name = 'NAIP' ), row = 1, col = 2 )

#  Add the Landsat 7 Image
fig_ov.add_trace( go.Image( z = ls7_rgb_image ), row=2, col=1 )
counts, bins = np.histogram( ls7_rgb_image.flatten(), bins = 30 )
bins = 0.5 * (bins[:-1] + bins[1:])

fig_ov.add_trace( go.Scatter( x = bins,
                              y = counts,
                              name = 'Landsat 7' ), row = 2, col = 2 )

#  Add the Landsat 9 Image
fig_ov.add_trace( go.Image( z = ls8_rgb_image ), row=3, col=1 )

counts, bins = np.histogram( ls8_rgb_image.flatten(), bins = 30 )
bins = 0.5 * (bins[:-1] + bins[1:])

fig_ov.add_trace( go.Scatter( x = bins,
                              y = counts,
                              name = 'Landsat 8' ), row = 3, col = 2 )

fig_ov.update_layout( width = 800,
                      height = 1200,
                      title  = 'Comparison of Landsat Imagery',
                      showlegend = False,
                      margin=dict(l=20, r=20, t=80, b=20, pad=2) )

fig_ov.update_xaxes(showticklabels=False)
fig_ov.update_yaxes(showticklabels=False)
fig_ov.show(renderer="svg")


As you can see, Landsat is pretty blurry with it's 30m Ground Sample Distance.